In [0]:
'''
  code by Minho Ryu @bzantium
  
'''
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for w, i in word_dict.items()}
n_class = len(word_dict)

# TextRNN Parameter
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])

    return input_batch, target_batch

# Model
class RNN(object):
    def __init__(self, sess, n_step, n_hidden, n_class):
        self.sess = sess
        self.n_step = n_step
        self.n_hidden = n_hidden
        self.n_class = n_class
        self._build_model()
        
    def _build_model(self):      
        self.X = tf.placeholder(tf.float32, [None, self.n_step, self.n_class]) # [batch_size, n_step, n_class]
        self.Y = tf.placeholder(tf.float32, [None, self.n_class])         # [batch_size, n_class]

        W = tf.Variable(tf.random_normal([self.n_hidden, self.n_class]))
        b = tf.Variable(tf.random_normal([self.n_class]))

        cell = tf.nn.rnn_cell.BasicRNNCell(self.n_hidden)
        outputs, states = tf.nn.dynamic_rnn(cell, self.X, dtype=tf.float32)

        # outputs : [batch_size, n_step, n_hidden]
        # states: [batch_size, n_hidden]
        logits = tf.matmul(states, W) + b # model : [batch_size, n_class]

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(0.001).minimize(self.cost)
        self.prediction = tf.cast(tf.argmax(logits, 1), tf.int32)

        tf.global_variables_initializer().run()
    
    def train(self, inputs, labels):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: inputs, self.Y: labels})
     
    def predict(self, inputs):
        return self.sess.run([self.prediction], feed_dict={self.X: inputs})
        
        

# Training
run_config = tf.ConfigProto()
run_config.gpu_options.allow_growth=True
with tf.Session(config=run_config) as sess:
    input_batch, target_batch = make_batch(sentences)
    model = RNN(sess, n_step, n_hidden, n_class)
    for epoch in range(5000):
        loss, _ = model.train(input_batch, target_batch)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    predictions =  model.predict(input_batch)[0]
    
for i, sentence in enumerate(sentences):
    print(sentence.split()[:2], '->', number_dict[predictions[i]])

Epoch: 1000 cost = 0.083354
Epoch: 2000 cost = 0.018689
Epoch: 3000 cost = 0.007426
Epoch: 4000 cost = 0.003606
Epoch: 5000 cost = 0.001924
['i', 'like'] -> dog
['i', 'love'] -> coffee
['i', 'hate'] -> milk
